[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_15.ipynb)

# Example 15: The Reference Paradox (Hypervolume Raw vs Ratio)

This example demonstrates the critical difference between the two hypervolume scaling modes in MoeaBench:
1. **'raw'**: The absolute physical volume dominated by the algorithm.
2. **'ratio'**: The relative efficiency compared to the maximum volume found in the current session (forces a 1.0 ceiling).

Understanding this difference is key to interpreting whether an algorithm is physically dominating more space, or simply changing its rank relative to a new competitor.

In [ ]:
# Install MoeaBench from GitHub (Uncomment if running in Colab)
# !pip install --quiet git+https://github.com/monacofj/moeabench.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from MoeaBench import mb
print(f"MoeaBench v{mb.system.version()}")

## 1. Setup the Problem and Experiments

We use **DTLZ2** and compare a baseline NSGA-II (small population) against a premium configuration (large population).

In [ ]:
dtlz2 = mb.mops.DTLZ2(n_var=7, n_obj=3)

print("Running Experiment 1 (Baseline: 20 individuals)...\n")
exp1 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=20, generations=50))
exp1.name = "Baseline NSGA-II"
exp1.run(repeat=5)

print("\nRunning Experiment 2 (Superior Competitor: 100 individuals)...\n")
exp2 = mb.experiment(dtlz2, mb.moeas.NSGA2(population=100, generations=50))
exp2.name = "Premium NSGA-II"
exp2.run(repeat=5)

## 2. The Paradox Demonstration

### Phase 1: Calculating Metrics with a Shared Bounding Box

To have absolute invariance, we must fix the Bounding Box (Nadir) for all. In a real session, MoeaBench does this by looking at everyone in the `ref` list.

In [ ]:
global_ref = [exp1, exp2]

# Raw HV: Physically Conquer (Invariant if BB is fixed)
hv_raw1 = mb.metrics.hv(exp1, ref=global_ref, scale='raw')
hv_raw2 = mb.metrics.hv(exp2, ref=global_ref, scale='raw')

# Ratio HV: Competitive Efficiency (Relative to best in session)
hv_ratio1 = mb.metrics.hv(exp1, ref=global_ref, scale='ratio')
hv_ratio2 = mb.metrics.hv(exp2, ref=global_ref, scale='ratio')

# Simulate the "Alone" case for exp1.
denom_alone = np.nanmax(hv_raw1.values[-1, :])
hv_ratio_alone1 = mb.metrics.MetricMatrix(hv_raw1.values / denom_alone, "Hypervolume (Ratio)")
hv_ratio_alone1.source_name = exp1.name

print(f"-> exp1 Ratio (Alone):    {hv_ratio_alone1.values[-1,:].mean():.4f} (Champion!)")
print(f"-> exp1 Ratio (vs exp2): {hv_ratio1.values[-1,:].mean():.4f} (Looks worse suddenly!)")
print(f"-> exp1 Raw (Alone):     {hv_raw1.values[-1,:].mean():.4f}")
print(f"-> exp1 Raw (vs exp2):    {hv_raw1.values[-1,:].mean():.4f} (STABLE!)")

## 3. Visual Comparison

We now plot the results side-by-side to visualize the concept of **Invariance** (Raw) vs **Fragility** (Ratio).

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 5))

# Plotting Raw (LEFT): Show the Invariance (Robustness)
mb.view.perf_history(hv_raw1, hv_raw1, ax=ax1, 
                     title="The Raw Invariance (Robustness)", 
                     labels=["exp1 (Standalone Reference)", "exp1 (Session Reference)"],
                     ylabel="Absolute Hypervolume")

# Plotting Ratio (RIGHT): Show the "Shift" (Fragility)
mb.view.perf_history(hv_ratio_alone1, hv_ratio1, ax=ax2, 
                     title="The Ratio Shift (Fragility)", 
                     labels=["exp1 (Baseline Only)", "exp1 (Competitive Session)"],
                     ylabel="Hypervolume (Ratio)")

plt.tight_layout()
plt.show()

## Conclusion

Compare the two plots:
1. **LEFT (Raw)**: The algorithm's contribution is physically invariant. Yes, there is only **one** curve visible because they overlap perfectly—they are 100% mathematically identical.
2. **RIGHT (Ratio)**: The same algorithm seems to lose quality just because someone else in the room is better.

**Scientific Note on the 'Coincidence'**:
In DTLZ benchmarks, the objective space is typically normalized to a unit hypercube (1.0x1.0x1.0). Thus, the physical volume (raw) and the percentage (ratio) converge to similar numbers. In a real-world engineering problem, these would be in completely different units and scales.

This demonstrates why **'raw' is the scientific default** in MoeaBench.